In [48]:
%load_ext autoreload
%autoreload 2

In [50]:
from nsp_all import NSP_all
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import sys
sys.argv = ['tx', '-n', 'nsp_alll', '-i', '192.168.30.6', '-p', '27263']

params = {
    "input_stream": "nsp_neural",
    "output_reref_stream": False,
    "reref_maxlen": 2000,
    "coefs_stream_name": "rereference_parameters",
    "reref_stream_idx": 0,
    "thresh_cross_stream_idx": 1,
    "band_power_stream_idx": 2,
    "binned_spikes_stream_idx": 3,
    "output_streams": ["reref_neural_a", "thresh_cross_a", "sbp_a", "binned_spikes_a"],
    "sync_key": ["tracking_id", "sync", "sync", "sync"],
    "ts_key": ["BRANDS_time", "ts", "ts", "ts"],
    "use_tracking_id": True,
    "td_type": "uint64",
    "sync_dict_key": "nsp_idx_1",
    "filt_stream": None,
    "adaptive_rms_stream": None,
    "pack_per_call": 1,
    "thresh_mult": -4.5,
    "thresh_calc_len": 2000,
    "butter_lowercut": 250,
    "butter_uppercut": None,
    "butter_order": 4,
    "enable_CAR": False,
    "output_filtered": False,
    "acausal_filter_lag": 120,
    "acausal_filter": "IIR",
    "ch_mask_stream": "z_mask_stream",
    "bandpower_logscale": False,
    "chan_per_stream": 256,  # Preserved as string since it's a reference
    "samp_per_stream": 30,
    "chan_total": 256,  # Preserved as string since it's a reference
    "start_channel": 0,
    "samp_freq": 30000,
    "output_dtype": "float32",
    "total_channels": "*total_features",  # Preserved as string since it's a reference
    "bin_size": 10,
    "bin_enable": True
}

node = NSP_all(params)

[nsp_alll] INFO: Coefficients entry found in Redis stream: rereference_parameters
[nsp_alll] INFO: Unshuffling matrix loaded from stream.
[nsp_alll] WARNING: 'ch_mask_stream' was set to z_mask_stream, but there were no entries. Defaulting to using all channels
[nsp_alll] INFO: Loading 4 order, 250 hz highpass acausal IIR-IIR filter
[nsp_alll] INFO: Loaded thresholds from the thresholds stream


[nsp_alll] Redis connection established on host: 192.168.30.6, port: 27263


In [5]:
node.profiler.print_stats()


Timing Statistics (in milliseconds):
--------------------------------------------------------------------------------
Operation                            Mean        Min        Max      Count
--------------------------------------------------------------------------------
INIT                                8.757      8.757      8.757          1
Redis read                          0.475      0.075      1.747      67557
Re-referencing                      0.101      0.099      0.237      67557
Filtering                           0.262      0.253      0.724      67553
Threshold crossing                  0.021      0.019      0.278      67553
Spike band power                    0.004      0.004      0.050      67553
Total Exec Time                     0.445      0.429      0.997      67553
Redis write                         0.132      0.113      0.683      67553
Total Time                          1.000      0.582      2.279      67553
Binning                             0.002      0.0

In [ ]:
node.run()

# verify data between new node and old streams

In [19]:
import json
import os
import numpy as np
import pandas as pd
from brand.redis import RedisLoggingHandler
from brand.timing import timespec_to_timestamp
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import logging
import time

from redis import ConnectionError, Redis
from brand import BRANDNode
from brand.redis import xread_sync
from threading import Timer

In [2]:


NAME = "timing_plots"
redis_host = '192.168.30.6'
redis_port = 27263
redis_socket = 10

r = Redis(redis_host, redis_port, retry_on_timeout=True)
r.ping()

supergraph_entry = r.xrevrange(b'supergraph_stream', '+', '-', 1)[0]

In [4]:
# n_entries = r.xlen("reref_neural_all")
print(r.xlen("binned_spikes"),r.xlen("binned_spikes_all"))

5715 5716


In [47]:
rn_all = r.xread({"reref_neural_all": b'0-0'}, count=1000)[0][1]
rn_old = r.xread({"reref_neural": b'0-0'}, count=1000)[0][1]
txa = r.xread({"thresh_cross": b'0-0'}, count=1000)[0][1]
tx4 = r.xread({"thresh_cross_4": b'0-0'}, count=1000)[0][1]
sba = r.xread({"sbp": b'0-0'}, count=1000)[0][1]
sb4 = r.xread({"sbp_3": b'0-0'}, count=1000)[0][1]
bs4 = r.xread({"binned_spikes": b'0-0'}, count=1000)[0][1]
bsa = r.xread({"binned_spikes_all": b'0-0'}, count=1000)[0][1]

In [ ]:
# verify reref_neural
for i in range(10000-100):
    if not rn_all[i][1][b'samples']==rn_old[i+4][1][b'samples']:
        print(0)

In [28]:
input_streams= ["thresh_cross_4", "sbp_3"]
stream_dict = {name.encode(): '$' for name in input_streams}
streams = xread_sync(r,stream_dict,block=0,
                        sync_field=b'timestamps',
                        sync_dtype='uint64',
                        count=10)
in_field={b"thresh_cross_4":b"crossings", b"sbp_3":b"samples"}
in_dtype={b"thresh_cross_4":np.dtype("int16"), b"sbp_3":np.dtype("float32")}
window =np.zeros((256,10),dtype=np.float32)


sync_entries = []
sync_entries_stream_dump={}
for stream in streams:
    stream_name, stream_entries = stream
    field = in_field[stream_name]
    in_type = in_dtype[stream_name]
    sync_entries_stream = []
    synx=[]
    for i, (entry_id, entry_dict) in enumerate(stream_entries):
        # load the input
        window[:, i] = np.frombuffer(entry_dict[field],
                                            dtype=in_type).astype(
                                                'float32')
        # log sync for this entry
        sync_entries_stream.append(
            json.loads(entry_dict[b'sync'].decode()))
        synx.append(json.loads(entry_dict[b'sync'].decode()))
    sync_entries_stream_dump[stream_name.decode()]=synx
    sync_entries.append(sync_entries_stream)
    # update the xread ID
    stream_dict[stream_name] = entry_id

# create sync dict from sync entries from input streams
sync_dict = {}
for stream in sync_entries:
    sync_entry_dict = stream[0]  # first entry from each stream
    for key in sync_entry_dict:
        sync_dict[key] = sync_entry_dict[key]
sync_dict_json = json.dumps(sync_dict)

In [46]:
# verify threshold crossing and binned threshould_crossing

cnt=0
for i in range(1000):
    idx,tx4_data =tx4[i]
    idx,txa_data = txa[i]
    idx,sb4_data =sb4[i]
    idx,sba_data = sba[i]
    idx,bs4_data =bs4[i]
    idx,bsa_data = bsa[i]
    p4=np.frombuffer(sb4_data[b'samples'],dtype=np.float32)
    pa=np.frombuffer(sba_data[b'samples'],dtype=np.float32)
    c4=np.frombuffer(tx4_data[b'crossings'],dtype=np.int16)
    ca=np.frombuffer(txa_data[b'crossings'],dtype=np.int16)
    b4=np.frombuffer(bs4_data[b'samples'],dtype=np.float32)[:256]
    ba=np.frombuffer(bsa_data[b'samples'],dtype=np.float32)[:256]
    if np.all(np.isclose(b4,ba)) and not np.all(np.isclose(p4,pa)) and np.all(np.isclose(c4,ca)):cnt+=1
if cnt:
    print(cnt)

995


In [ ]:
buffer_num=0
cross_bin_buffer4 = np.zeros((256,10),np.int16)
cross_bin_buffera = np.zeros((256,10),np.int16)
ak=5
for i in range(10):

    cross_bin_buffer4[:, i] = np.frombuffer(tx4[5+i][1][b'crossings'],dtype=np.int16)
    cross_bin_buffera[:, i] = np.frombuffer(txa[+i][1][b'crossings'],dtype=np.int16)


aa=cross_bin_buffera.sum(axis=1)
ba=np.frombuffer(bsa[1][1][b'samples'],dtype=np.float32)[:256]

a4=cross_bin_buffer4.sum(axis=1)
b4=np.frombuffer(bs4[0][1][b'samples'],dtype=np.float32)[:256]

print(np.all(aa==ba),np.all(a4==b4),np.all(aa==a4),np.all(ba==b4))

Timing Statistics (in milliseconds):
--------------------------------------------------------------------------------
Operation                            Mean        Min        Max      Count
--------------------------------------------------------------------------------
INIT                               13.231     13.231     13.231          1
Redis read                          0.141      0.071      4.358     134727
Re-referencing                      0.155      0.150      0.326     134727
Filtering                           0.261      0.251      1.021     134717
Threshold crossing                  0.287      0.274      0.458     134717
Spike band power                    0.004      0.003      0.027     134717
Total Exec Time                     0.760      0.733      1.659     134717
Redis write                         0.148      0.108      0.837     134717
Total Time                          1.000      0.884      5.228     134717
Binning                             0.002      0.001      0.022      13470
Total Binning Time                 10.001      9.208     15.749      13470

In [12]:
##NUMBA thresh
node.profiler.print_stats()


Timing Statistics (in milliseconds):
--------------------------------------------------------------------------------
Operation                            Mean        Min        Max      Count
--------------------------------------------------------------------------------
INIT                               13.231     13.231     13.231          1
Redis read                          0.141      0.071      4.358     134727
Re-referencing                      0.155      0.150      0.326     134727
Filtering                           0.261      0.251      1.021     134717
Threshold crossing                  0.287      0.274      0.458     134717
Spike band power                    0.004      0.003      0.027     134717
Total Exec Time                     0.760      0.733      1.659     134717
Redis write                         0.148      0.108      0.837     134717
Total Time                          1.000      0.884      5.228     134717
Binning                             0.002      0.0